In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
import datetime
import time
from collections import Counter
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from intervals import Interval
import warnings
warnings.filterwarnings('ignore')

## 垃圾数据处理

In [2]:
# 数据预处理
data=pd.read_csv('./data/Krommenie_OpenLitterMap.csv')

In [3]:
df = data.fillna(0)# 填充缺失值

In [4]:
df

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,...,ear_plugs,elec_small,elec_large,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3
0,70424,2,iPhone 6,2019-07-30 14:23:20,2020-03-08 20:24:37,52.495358,4.754881,Yes,"202, Provincialeweg, Krommenie, Uitgeest, Nort...",2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,72163,2,iPhone 6,2019-08-31 16:11:46,2020-03-13 20:06:40,52.495617,4.750478,Yes,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,76013,2,iPhone 6,2019-08-28 18:20:03,2020-05-18 10:17:44,52.495475,4.755081,No,"202, Provincialeweg, Krommenie, Uitgeest, Nort...",2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,76153,2,iPhone 6,2020-03-26 18:05:22,2020-05-18 10:47:37,52.497681,4.747939,No,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,76170,2,iPhone 6,2020-04-05 20:37:25,2020-05-18 10:49:26,52.497714,4.747989,No,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16441,368173,2,iPhone 6,2023-01-10 13:30:11,2023-01-18 20:26:40,52.499264,4.748108,Yes,"17, Venusstraat, Krommenie, Zaanstad, North Ho...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,category:fireworks,object:fireworks,0
16442,368174,2,iPhone 6,2023-01-10 13:30:23,2023-01-18 20:26:42,52.499314,4.748144,Yes,"17, Venusstraat, Krommenie, Zaanstad, North Ho...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,category:fireworks,object:fireworks,0
16443,368795,2,iPhone XR,2023-01-13 13:39:28,2023-01-21 21:20:46,52.503536,4.749461,Yes,"Marslaan, Krommenie, Zaanstad, North Holland, ...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,category:fireworks,object:fireworks,0
16444,368796,2,iPhone XR,2023-01-13 13:39:30,2023-01-21 21:20:48,52.503536,4.749461,Yes,"Marslaan, Krommenie, Zaanstad, North Holland, ...",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,category:fireworks,object:fireworks,0


In [5]:
sum(df['total_litter'])

16986.0

### 垃圾分类

In [5]:
# 垃圾分类
def sumc(x):# 定义合并类的函数
    count_sum = 0
    for k in x.iteritems():
        count_sum += k[1]
    return count_sum

In [6]:
df['SMOKING']=df.loc[:,'butts':'smokingOther'].apply(sumc,axis=1)
df['SMOKING'].sum()

2135.0

In [7]:
df=df.drop(df.loc[:,'butts':'smokingOther'],axis=1)

In [8]:
df['FOOD']=df.loc[:,'sweetWrappers':'chewing_gum'].apply(sumc,axis=1)
df['FOOD'].sum()

2667.0

In [9]:
df=df.drop(df.loc[:,'sweetWrappers':'chewing_gum'],axis=1)

In [10]:
df['COFFEE']=df.loc[:,'coffeeCups':'coffeeOther'].apply(sumc,axis=1)
df['COFFEE'].sum()

358.0

In [11]:
df=df.drop(df.loc[:,'coffeeCups':'coffeeOther'],axis=1)

In [12]:
df['ALCOHOL']=df.loc[:,'beerCan':'alcoholOther'].apply(sumc,axis=1)
df['ALCOHOL'].sum()

839.0

In [13]:
df=df.drop(df.loc[:,'beerCan':'alcoholOther'],axis=1)

In [14]:
df['SOFTDRINKS']=df.loc[:,'waterBottle':'softDrinkOther'].apply(sumc,axis=1)
df['SOFTDRINKS'].sum()

5014.0

In [15]:
df=df.drop(df.loc[:,'waterBottle':'softDrinkOther'],axis=1)

In [16]:
df['SANITARY']=df.loc[:,'condoms':'hand_sanitiser'].apply(sumc,axis=1)
df['SANITARY'].sum()

557.0

In [17]:
df=df.drop(df.loc[:,'condoms':'hand_sanitiser'],axis=1)

In [18]:
df['COASTAL']=df.loc[:,'microplastics':'styro_large'].apply(sumc,axis=1)
df['COASTAL'].sum()

25.0

In [19]:
df=df.drop(df.loc[:,'microplastics':'styro_large'],axis=1)

In [20]:
df['DUMPING']=df.loc[:,'small':'large'].apply(sumc,axis=1)
df['DUMPING'].sum()

0.0

In [21]:
df=df.drop(df.loc[:,'small':'large'],axis=1)

In [22]:
df['INDUSTRIAL']=df.loc[:,'oil':'industrial_other'].apply(sumc,axis=1)
df['INDUSTRIAL'].sum()

1.0

In [23]:
df['BRANDS']=df.loc[:,'aadrink':'wrigleys'].apply(sumc,axis=1)
df['BRANDS'].sum()

4303.0

In [24]:
df=df.drop(df.loc[:,'aadrink':'wrigleys'],axis=1)

In [25]:
df=df.drop(df.loc[:,'oil':'industrial_other'],axis=1)

In [26]:
df['DOGSHIT']=df.loc[:,'poo':'poo_in_bag'].apply(sumc,axis=1)
df['DOGSHIT'].sum()

12.0

In [27]:
df=df.drop(df.loc[:,'poo':'poo_in_bag'],axis=1)

In [28]:
df['ART']=df.loc[:,'item']

In [29]:
df['ART'].sum()

0.0

In [30]:
df['MATERIAL']=df.loc[:,'aluminium':'wood'].apply(sumc,axis=1)

In [31]:
df['MATERIAL'].sum()

804.0

In [32]:
df=df.drop(df.loc[:,'aluminium':'wood'],axis=1)

In [33]:
df=df.drop('item',axis=1)

In [34]:
df['OTHER']=df.loc[:,'random_litter':'other'].apply(sumc,axis=1)
df['OTHER'].sum()

4514.0

In [35]:
df=df.drop(df.loc[:,'random_litter':'other'],axis=1)

In [36]:
df.columns

Index(['id', 'verification', 'phone', 'date_taken', 'date_uploaded', 'lat',
       'lon', 'picked up', 'address', 'total_litter', 'SMOKING', 'FOOD',
       'COFFEE', 'ALCOHOL', 'SOFTDRINKS', 'SANITARY', 'COASTAL', 'DUMPING',
       'INDUSTRIAL', 'BRANDS', 'DOGSHIT', 'ART', 'MATERIAL', 'OTHER',
       'custom_tag_1', 'custom_tag_2', 'custom_tag_3'],
      dtype='object')

In [37]:
df=df.drop(df.loc[:,'custom_tag_1':'custom_tag_3'],axis=1)# 删除不需要的标签

In [38]:
df['OTHER']=df['OTHER']+df['ART']+df['DOGSHIT']+df['INDUSTRIAL']+df['DUMPING']+df['COASTAL']

In [39]:
df['OTHER'].sum()

4552.0

In [40]:
df=df.drop(['ART','DOGSHIT','INDUSTRIAL','DUMPING','COASTAL'],axis=1)

In [41]:
df # 分类完毕

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER
0,70424,2,iPhone 6,2019-07-30 14:23:20,2020-03-08 20:24:37,52.495358,4.754881,Yes,"202, Provincialeweg, Krommenie, Uitgeest, Nort...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,72163,2,iPhone 6,2019-08-31 16:11:46,2020-03-13 20:06:40,52.495617,4.750478,Yes,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,76013,2,iPhone 6,2019-08-28 18:20:03,2020-05-18 10:17:44,52.495475,4.755081,No,"202, Provincialeweg, Krommenie, Uitgeest, Nort...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,76153,2,iPhone 6,2020-03-26 18:05:22,2020-05-18 10:47:37,52.497681,4.747939,No,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,76170,2,iPhone 6,2020-04-05 20:37:25,2020-05-18 10:49:26,52.497714,4.747989,No,"Provincialeweg, Krommenie, Uitgeest, North Hol...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16441,368173,2,iPhone 6,2023-01-10 13:30:11,2023-01-18 20:26:40,52.499264,4.748108,Yes,"17, Venusstraat, Krommenie, Zaanstad, North Ho...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16442,368174,2,iPhone 6,2023-01-10 13:30:23,2023-01-18 20:26:42,52.499314,4.748144,Yes,"17, Venusstraat, Krommenie, Zaanstad, North Ho...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16443,368795,2,iPhone XR,2023-01-13 13:39:28,2023-01-21 21:20:46,52.503536,4.749461,Yes,"Marslaan, Krommenie, Zaanstad, North Holland, ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16444,368796,2,iPhone XR,2023-01-13 13:39:30,2023-01-21 21:20:48,52.503536,4.749461,Yes,"Marslaan, Krommenie, Zaanstad, North Holland, ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
df=df[-df['date_taken'].str.contains('2019')].reset_index(drop=True) # 删除2019年的零散数据

In [43]:
df=df[-df['date_taken'].str.contains('2020')].reset_index(drop=True) # 删除2020年的零散数据

In [44]:
df=df[-df['date_taken'].str.contains('2021')].reset_index(drop=True) # 删除2021年的零散数据

In [45]:
df=df[-df['date_taken'].str.contains('2023')].reset_index(drop=True) # 删除2023年的零散数据

In [46]:
df=df.drop(columns=['date_uploaded'])

In [47]:
df=df.rename(columns={'date_taken':'datetime'})

In [48]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER
0,240112,2,iPhone 6,2022-01-02 14:58:19,52.493825,4.771028,Yes,"Provincialeweg, Krommenie, Zaanstad, North Hol...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,240113,2,iPhone 6,2022-01-02 14:51:17,52.495187,4.758970,Yes,"Provincialeweg, Krommenie, Zaanstad, North Hol...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,240116,2,iPhone 6,2022-01-02 14:58:19,52.493825,4.771028,Yes,"Provincialeweg, Krommenie, Zaanstad, North Hol...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,240118,2,iPhone 6,2022-01-02 14:52:03,52.495025,4.760038,Yes,"62A, Provincialeweg, Krommenie, Zaanstad, Nort...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,240119,2,iPhone 6,2022-01-01 12:04:40,52.504005,4.749692,Yes,"11, Marslaan, Krommenie, Zaanstad, North Holla...",1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10798,365679,2,iPhone 6,2022-12-30 12:40:10,52.495064,4.757611,Yes,"Teunis Glijnisweg, Krommenie, Zaanstad, North ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10799,365680,2,iPhone 6,2022-12-30 13:04:44,52.495036,4.758000,Yes,"Teunis Glijnisweg, Krommenie, Zaanstad, North ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10800,365681,2,iPhone 6,2022-12-30 13:04:50,52.495047,4.758008,Yes,"Teunis Glijnisweg, Krommenie, Zaanstad, North ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10801,365716,2,iPhone 6,2022-12-30 12:40:18,52.495064,4.757625,Yes,"Teunis Glijnisweg, Krommenie, Zaanstad, North ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
df.to_csv('./outdata/Krommenie_preliminary.csv')

### 合并时间

In [50]:
# df=df.rename(columns={'date_taken':'datetime'})

In [51]:
df['datetime']=pd.to_datetime(df['datetime'])

In [52]:
df['datetime']

0       2022-01-02 14:58:19
1       2022-01-02 14:51:17
2       2022-01-02 14:58:19
3       2022-01-02 14:52:03
4       2022-01-01 12:04:40
                ...        
10798   2022-12-30 12:40:10
10799   2022-12-30 13:04:44
10800   2022-12-30 13:04:50
10801   2022-12-30 12:40:18
10802   2022-12-27 12:18:11
Name: datetime, Length: 10803, dtype: datetime64[ns]

In [53]:
Date=df['datetime'].dt.date
df_dateconcat=df.loc[:,'total_litter':'OTHER'].groupby(Date).sum() # 合并同日期的垃圾数量
df_dateconcat

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER
datetime,,,,,,,,,,
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0
2022-01-07,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0
2022-01-10,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0
2022-01-11,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
2022-12-25,229.0,8.0,29.0,12.0,67.0,65.0,1.0,111.0,12.0,35.0
2022-12-27,152.0,6.0,49.0,7.0,4.0,32.0,4.0,31.0,5.0,45.0
2022-12-29,160.0,4.0,36.0,4.0,14.0,30.0,1.0,25.0,4.0,67.0


In [54]:
df_dateconcat.max()

total_litter    466.0
SMOKING         290.0
FOOD            102.0
COFFEE           12.0
ALCOHOL          67.0
SOFTDRINKS       86.0
SANITARY         34.0
BRANDS          156.0
MATERIAL         19.0
OTHER           193.0
dtype: float64

### 对缺失的日期补0（舍弃）

In [53]:
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2022, 12, 31)
date_range = pd.date_range(start, end)

In [54]:
date_range

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
               '2022-01-09', '2022-01-10',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [55]:
# 筛选缺失的日期
df_date=pd.to_datetime(df_dateconcat.index) 
dateflag=date_range.isin(df_date) 

In [56]:
# date_all=np.concatenate((pd.Series(df_dateconcat.index).values,pd.Series(date_range[~dateflag].date).values),axis=0)# 合并两个时间
# date_all.size
# date_all与date_range相同，弃用data_all

In [57]:
Counter(dateflag)

Counter({True: 249, False: 116})

In [58]:
# 缺失的数据
dfdif=pd.DataFrame(np.zeros(116*10).reshape(116,10),
                   index=[pd.Series(date_range[~dateflag].date).values],
                   columns=['total_litter','SMOKING','FOOD','COFFEE','ALCOHOL','SOFTDRINKS','SANITARY','BRANDS','MATERIAL','OTHER'])

In [59]:
df_dateall=pd.concat([df_dateconcat,dfdif])# 合并

In [60]:
df_dateall=df_dateall.loc[:,'total_litter':'OTHER'].groupby(date_range).sum()

In [138]:
# df_dateall['datetime']=df_dateall.index

In [146]:
df_dateall

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,datetime
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,2022-01-01
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,2022-01-02
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2022-01-03
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,2022-01-04
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30


In [145]:
# df_dateall.to_csv('./outdata/Krommenie_alldaylitter.csv')

In [62]:
df_dateall.max()

total_litter    466.0
SMOKING         290.0
FOOD            102.0
COFFEE           12.0
ALCOHOL          67.0
SOFTDRINKS       86.0
SANITARY         34.0
BRANDS          156.0
MATERIAL         19.0
OTHER           193.0
dtype: float64

## 天气时间数据处理

In [55]:
weather = pd.read_csv('./data/Krommenie_weather/Krommenie_weather.csv')

In [56]:
weather['date']=pd.to_datetime(weather['date'])

In [57]:
weather['weather'].value_counts()

Sunny      214
Cloudy      74
Showers     57
Foggy       20
Name: weather, dtype: int64

In [58]:
wtcdt=pd.read_csv('./data/Krommenie_weather/Krommenie.csv')

In [59]:
wtcdt['date']=pd.to_datetime(wtcdt['date'])

In [60]:
wtcdt = pd.DataFrame(wtcdt).set_index('date') # 将时间作为索引

In [61]:
wtcdt

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2022-01-01,54,52,50,50,88,88,12,12
2022-01-02,55,50,50,45,94,77,30,12
2022-01-03,52,46,46,45,93,76,25,10
2022-01-04,48,37,46,32,93,70,16,5
2022-01-05,46,37,41,30,93,70,30,10
...,...,...,...,...,...,...,...,...
2022-12-27,46,41,43,36,93,71,22,9
2022-12-28,52,43,50,41,100,87,30,20
2022-12-29,52,45,48,41,93,76,29,15


### 将天气状况加入补全后数据（舍弃）

In [ ]:
M_test=pd.concat([df_dateall,wtcdt],axis=1)# 横向合并

In [71]:
M_test

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,54,52,50,50,88,88,12,12
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,55,50,50,45,94,77,30,12
2022-01-03,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,52,46,46,45,93,76,25,10
2022-01-04,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,48,37,46,32,93,70,16,5
2022-01-05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,46,37,41,30,93,70,30,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46,41,43,36,93,71,22,9
2022-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,43,50,41,100,87,30,20
2022-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,45,48,41,93,76,29,15
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54,39,52,39,100,81,29,12


### 将天气状况加入未补全数据

In [63]:
# 匹配时间
df_dateconcat.index

Index([2022-01-01, 2022-01-02, 2022-01-07, 2022-01-10, 2022-01-11, 2022-01-12,
       2022-01-13, 2022-01-15, 2022-01-17, 2022-01-18,
       ...
       2022-12-18, 2022-12-19, 2022-12-21, 2022-12-23, 2022-12-24, 2022-12-25,
       2022-12-27, 2022-12-29, 2022-12-30, 2022-12-31],
      dtype='object', name='datetime', length=249)

In [64]:
isindex1=wtcdt.index.isin(df_dateconcat.index)

In [65]:
wtcdt[isindex1]

,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
date,,,,,,,,
2022-01-01,54,52,50,50,88,88,12,12
2022-01-02,55,50,50,45,94,77,30,12
2022-01-07,45,36,39,32,93,70,24,5
2022-01-10,45,32,41,30,100,81,9,2
2022-01-11,41,32,39,32,100,76,14,6
...,...,...,...,...,...,...,...,...
2022-12-25,48,45,48,45,100,93,12,5
2022-12-27,46,41,43,36,93,71,22,9
2022-12-29,52,45,48,41,93,76,29,15


In [66]:
isindex2=weather['date'].isin(df_dateconcat.index)

In [67]:
isindex2.value_counts()

True     249
False    116
Name: date, dtype: int64

In [68]:
wtdf = pd.DataFrame(weather[isindex2]).set_index('date')

In [69]:
M=pd.concat([df_dateconcat,wtdf,wtcdt[isindex1]],axis=1)# 横向合并

In [70]:
M

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,BRANDS,MATERIAL,OTHER,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed
2022-01-01,57.0,1.0,0.0,2.0,12.0,32.0,4.0,28.0,0.0,6.0,Showers,54,52,50,50,88,88,12,12
2022-01-02,48.0,1.0,2.0,0.0,7.0,17.0,10.0,20.0,0.0,11.0,Showers,55,50,50,45,94,77,30,12
2022-01-07,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,Showers,45,36,39,32,93,70,24,5
2022-01-10,41.0,0.0,0.0,0.0,1.0,36.0,3.0,22.0,0.0,1.0,Cloudy,45,32,41,30,100,81,9,2
2022-01-11,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,Foggy,41,32,39,32,100,76,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,229.0,8.0,29.0,12.0,67.0,65.0,1.0,111.0,12.0,35.0,Showers,48,45,48,45,100,93,12,5
2022-12-27,152.0,6.0,49.0,7.0,4.0,32.0,4.0,31.0,5.0,45.0,Cloudy,46,41,43,36,93,71,22,9
2022-12-29,160.0,4.0,36.0,4.0,14.0,30.0,1.0,25.0,4.0,67.0,Showers,52,45,48,41,93,76,29,15
2022-12-30,228.0,29.0,30.0,12.0,29.0,38.0,1.0,52.0,9.0,80.0,Showers,54,39,52,39,100,81,29,12


In [67]:
# M_train=M.drop(M.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

### 对天气描述编码

In [68]:
weather

,date,weather
0,2022-01-01,Showers
1,2022-01-02,Showers
2,2022-01-03,Sunny
3,2022-01-04,Showers
4,2022-01-05,Showers
...,...,...
360,2022-12-27,Cloudy
361,2022-12-28,Showers
362,2022-12-29,Showers
363,2022-12-30,Showers


In [69]:
weather = pd.DataFrame(weather).set_index('date') # 将时间作为索引

In [71]:
wtdf

,weather
date,
2022-01-01,Showers
2022-01-02,Showers
2022-01-07,Showers
2022-01-10,Cloudy
2022-01-11,Foggy
...,...
2022-12-25,Showers
2022-12-27,Cloudy
2022-12-29,Showers


In [72]:
values_wt=wtdf['weather']

In [73]:
# 整数编码
label_encoder_wt = LabelEncoder()
integer_encoded_wt = label_encoder_wt.fit_transform(values_wt)
integer_encoded_wt

array([2, 2, 2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 2, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 0, 2, 0, 3, 0, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 0, 3,
       3, 3, 2, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 2, 0, 3, 3, 3, 3, 3,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 3, 3, 3, 0,
       3, 0, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3,
       2, 2, 1, 3, 0, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 3, 3, 3, 3, 2, 2, 2,
       1, 3, 3, 2, 3, 0, 3, 3, 1, 0, 0, 2, 2, 0, 1, 3, 3, 3, 1, 3, 2, 1,
       2, 3, 2, 0, 2, 2, 2])

In [74]:
# 二进制编码
wt_encoded = OneHotEncoder(sparse=False)
integer_encoded_wt = integer_encoded_wt.reshape(len(integer_encoded_wt), 1)
wt_encoded = wt_encoded.fit_transform(integer_encoded_wt)
wt_encoded.shape

(249, 4)

In [75]:
# 第一个例子的反转
inverted_wt = label_encoder_wt.inverse_transform([argmax(wt_encoded[0, :])])
inverted_wt

array(['Showers'], dtype=object)

### 天气特征归一化(舍弃)

In [78]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [79]:
M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1)

array([[36.59],
       [34.  ],
       [39.  ],
       ...,
       [36.  ],
       [27.  ],
       [30.58]])

In [80]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M_train.loc[:,'Actual Mean Temperature':'Actual Sea Level Pressure'].values.reshape(-1, 1))

In [81]:
mm_weather=mm_data.reshape(228,10)
mm_weather

array([[0.43559524, 0.4047619 , 0.46428571, ..., 0.42857143, 0.38095238,
        0.35809524],
       [0.44690476, 0.4047619 , 0.48809524, ..., 0.44047619, 0.38095238,
        0.35666667],
       [0.42809524, 0.35714286, 0.48809524, ..., 0.42857143, 0.35714286,
        0.35869048],
       ...,
       [0.37857143, 0.32142857, 0.51190476, ..., 0.38095238, 0.32142857,
        0.36154762],
       [0.47130952, 0.42857143, 0.51190476, ..., 0.44047619, 0.33333333,
        0.36333333],
       [0.49380952, 0.42857143, 0.57142857, ..., 0.42857143, 0.32142857,
        0.36404762]])

In [232]:
# 合并归一化后的数据
mm_weather_df=pd.DataFrame(mm_weather)
mm_weather_df=mm_weather_df.set_index(wtdf.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(wtdf.index)
ML=pd.concat([wt_encoded_df,mm_weather_df,M_train['total_litter']],axis=1)

In [233]:
ML

,0,1,2,3,0,1,2,3,4,5,6,7,8,9,total_litter
2021-01-06,0.0,0.0,0.0,1.0,0.435595,0.404762,0.464286,0.369048,0.476190,0.178571,0.388214,0.428571,0.380952,0.358095,3.0
2021-01-07,0.0,0.0,1.0,0.0,0.446905,0.404762,0.488095,0.369048,0.476190,0.190476,0.420595,0.440476,0.380952,0.356667,2.0
2021-01-08,0.0,0.0,1.0,0.0,0.428095,0.357143,0.488095,0.369048,0.476190,0.083333,0.405833,0.428571,0.357143,0.358690,114.0
2021-01-09,0.0,1.0,0.0,0.0,0.386071,0.297619,0.511905,0.369048,0.476190,0.095238,0.352024,0.440476,0.297619,0.360476,58.0
2021-01-10,1.0,0.0,0.0,0.0,0.431429,0.333333,0.488095,0.369048,0.476190,0.142857,0.393690,0.440476,0.321429,0.360476,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,0.0,0.0,1.0,0.0,0.536548,0.511905,0.547619,0.392857,0.500000,0.142857,0.518690,0.535714,0.488095,0.364762,92.0
2021-12-20,1.0,0.0,0.0,0.0,0.495595,0.380952,0.535714,0.392857,0.500000,0.083333,0.418690,0.464286,0.357143,0.361905,232.0
2021-12-21,0.0,0.0,0.0,1.0,0.378571,0.321429,0.511905,0.392857,0.500000,0.059524,0.339762,0.380952,0.321429,0.361548,14.0
2022-01-20,1.0,0.0,0.0,0.0,0.471310,0.428571,0.511905,0.369048,0.476190,0.357143,0.392024,0.440476,0.333333,0.363333,5.0


In [84]:
ML_array=np.array(ML)

In [85]:
np.save('Hoorn_array', ML)

## poi分类统计（舍弃）

In [6]:
points=gpd.read_file('./data/Krommenie_shape/points.shp')

In [7]:
points

,osm_id,timestamp,name,type,geometry
0,46750399,None,None,speed_camera,POINT (4.76606 52.49424)
1,46776504,None,None,traffic_signals,POINT (4.73953 52.50258)
2,262846795,None,None,crossing,POINT (4.75409 52.49316)
3,263313168,None,None,crossing,POINT (4.75551 52.49743)
4,434050121,None,None,traffic_signals,POINT (4.77377 52.49361)
...,...,...,...,...,...
433,1497423568,None,None,museum,POINT (4.73558 52.50194)
434,1502077603,None,IKC Toermalijn,school,POINT (4.79343 52.49740)
435,1502114206,None,Suri Rai,fast_food,POINT (4.79060 52.49293)
436,1502129852,None,KDV De Kabouters,childcare,POINT (4.75557 52.50243)


In [117]:
public=points['type'].isin(['post_box', 'pub',  'school', 'social_facility','charging_station','childcare','arts_centre',

'community_centre','kindergarten','library',  'elevator','fire_station','post_depot', 

 'social_centre', 'marketplace','police', 'theatre','yard','music_school','townhall'])

In [118]:
poi_public=points['type'][public].value_counts()

In [119]:
poi_public=poi_public.values.sum()

In [120]:
poi_public

53

In [121]:
points['type'][public]='public'

In [122]:
health=points['type'].isin(['dentist','doctors','pharmacy', 'veterinary','hospital', 'clinic'])

In [123]:
poi_health=points['type'][health].value_counts()

In [124]:
poi_health=poi_health.values.sum()

In [125]:
points['type'][health]='health'

In [126]:
leisure=points['type'].isin(['cafe', 'cinema','casino'])

In [127]:
poi_leisure=points['type'][leisure].value_counts()

In [128]:
poi_leisure=poi_leisure.values.sum()

In [129]:
poi_leisure

8

In [130]:
points['type'][leisure]='leisure'

In [131]:
catering=points['type'].isin([ 'restaurant','fast_food'])

In [132]:
poi_catering=points['type'][catering].value_counts()

In [133]:
poi_catering=poi_catering.values.sum()

In [134]:
poi_catering

28

In [135]:
points['type'][catering]='catering'

In [136]:
accommodation=points['type'].isin(['hotel','shelter','caravan_site'])

In [137]:
poi_accommodation=points['type'][accommodation].value_counts()

In [138]:
poi_accommodation=poi_accommodation.values.sum()

In [139]:
poi_accommodation

0

In [140]:
points['type'][accommodation]='accommodation'

In [141]:
shopping=points['type'].isin(['car_wash','vending_machine', 'car_sharing','ice_cream','parking', 'shower','bicycle_repair_s', 'car_rental','taxi'])

In [142]:
poi_shopping=points['type'][shopping].value_counts()

In [143]:
poi_shopping=poi_shopping.values.sum()

In [144]:
poi_shopping

6

In [145]:
points['type'][shopping]='shopping'

In [146]:
money=points['type'].isin(['atm','bank'])

In [147]:
poi_money=points['type'][money].value_counts()

In [148]:
poi_money=poi_money.values.sum()

In [149]:
poi_money

4

In [150]:
points['type'][money]='money'

In [151]:
tourism=points['type'].isin(['artwork', 'memorial', 'gallery','museum','attraction', 'cannon',  'archaeological_s',  'viewpoint', 'picnic_site'])

In [152]:
poi_tourism=points['type'][tourism].value_counts()

In [153]:
poi_tourism=poi_tourism.values.sum()

In [154]:
poi_tourism

7

In [155]:
points['type'][tourism]='tourism'

In [156]:
miscpoi=points['type'].isin(['bench','waste_basket','tower','toilets','surveillance','works','drinking_water', 'windmill','fountain', 'water_well'])

In [157]:
poi_miscpoi=points['type'][miscpoi].value_counts()

In [158]:
poi_miscpoi=poi_miscpoi.values.sum()

In [159]:
poi_miscpoi

34

In [160]:
points['type'][miscpoi]='miscpoi'

In [161]:
other=points['type'].isin(['construction','give_way','clock','street_cabinet', 'studio', 'place_of_worship',  'crane',  'communications_t','flagpole', 'chimney', 'lighthouse','tram_stop', 'yes'])

In [162]:
poi_other=points['type'][other].value_counts()

In [163]:
poi_other=poi_other.values.sum()

In [164]:
poi_other

4

In [165]:
points['type'][other]='other'

In [166]:
traffic=points['type'].isin(['mast','bicycle_parking','crossing', 'recycling', 'traffic_signals', 'signal','switch', 'level_crossing','motorway_junctio','ferry_terminal','turning_circle', 'buffer_stop', 'street_lamp','fuel', 'parking_entrance', 'stop', 'station','milestone','mini_roundabout',  'railway_crossing','junction'])

In [167]:
poi_traffic=points['type'][traffic].value_counts()

In [168]:
poi_traffic=poi_traffic.values.sum()

In [169]:
poi_traffic

207

In [170]:
points['type'][traffic]='traffic'

In [182]:
poi=pd.DataFrame(np.array([poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other]).reshape(1,10),
                 columns=['poi_public','poi_health','poi_leisure','poi_catering','poi_shopping','poi_money','poi_tourism','poi_miscpoi','poi_traffic','poi_other'],
                 index=['counter']
                )# 去除数量为0的poi

In [183]:
poi

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
counter,53,17,8,28,6,4,7,34,207,4


In [173]:
# poi_other=poi_other+poi_money+poi_shopping+poi_accommodation+poi_health+poi_leisure

In [144]:
# poi['poi_other']=poi_other

In [145]:
# poi=poi.drop(['poi_health','poi_leisure','poi_accommodation','poi_shopping','poi_money'],axis=1)

In [146]:
# poi

In [184]:
poi_array=np.array(poi)

In [185]:
poi_array=np.tile(poi,(249,1))

In [186]:
poi_array

array([[ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       ...,
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4],
       [ 53,  17,   8, ...,  34, 207,   4]], dtype=int64)

In [187]:
poi_df=pd.DataFrame(poi_array,columns=poi.columns,index=M_train.index)

In [188]:
poi_df

,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,53,17,8,28,6,4,7,34,207,4
2022-01-02,53,17,8,28,6,4,7,34,207,4
2022-01-07,53,17,8,28,6,4,7,34,207,4
2022-01-10,53,17,8,28,6,4,7,34,207,4
2022-01-11,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...
2022-12-25,53,17,8,28,6,4,7,34,207,4
2022-12-27,53,17,8,28,6,4,7,34,207,4
2022-12-29,53,17,8,28,6,4,7,34,207,4
2022-12-30,53,17,8,28,6,4,7,34,207,4


In [189]:
M_train=pd.concat([M_train,poi_df],axis=1)

In [190]:
M_train

,total_litter,weather,Max Temperature,Min Temperature,Max Dew Point,Min Dew Point,Max Humidity,Min Humidity,Max Wind Speed,Min Wind Speed,poi_public,poi_health,poi_leisure,poi_catering,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2022-01-01,57.0,Showers,54,52,50,50,88,88,12,12,53,17,8,28,6,4,7,34,207,4
2022-01-02,48.0,Showers,55,50,50,45,94,77,30,12,53,17,8,28,6,4,7,34,207,4
2022-01-07,2.0,Showers,45,36,39,32,93,70,24,5,53,17,8,28,6,4,7,34,207,4
2022-01-10,41.0,Cloudy,45,32,41,30,100,81,9,2,53,17,8,28,6,4,7,34,207,4
2022-01-11,2.0,Foggy,41,32,39,32,100,76,14,6,53,17,8,28,6,4,7,34,207,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,229.0,Showers,48,45,48,45,100,93,12,5,53,17,8,28,6,4,7,34,207,4
2022-12-27,152.0,Cloudy,46,41,43,36,93,71,22,9,53,17,8,28,6,4,7,34,207,4
2022-12-29,160.0,Showers,52,45,48,41,93,76,29,15,53,17,8,28,6,4,7,34,207,4
2022-12-30,228.0,Showers,54,39,52,39,100,81,29,12,53,17,8,28,6,4,7,34,207,4


In [191]:
M_train.columns

Index(['total_litter', 'weather', 'Max Temperature', 'Min Temperature',
       'Max Dew Point', 'Min Dew Point', 'Max Humidity', 'Min Humidity',
       'Max Wind Speed', 'Min Wind Speed', 'poi_public', 'poi_health',
       'poi_leisure', 'poi_catering', 'poi_shopping', 'poi_money',
       'poi_tourism', 'poi_miscpoi', 'poi_traffic', 'poi_other'],
      dtype='object')

## 将数值型数据归一化

In [76]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [77]:
# M_test=M_test.drop(M_test.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

In [78]:
M = M.drop(columns=['weather'])

In [79]:
M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1)

array([[ 1.],
       [ 0.],
       [ 2.],
       ...,
       [63.],
       [31.],
       [ 9.]])

In [80]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M.loc[:,'SMOKING':'Min Wind Speed'].values.reshape(-1,1))

In [81]:
mm_wp=mm_data.reshape(249,17)

In [82]:
Counter(mm_wp.flatten())

Counter({0.0034482758620689655: 394,
         0.0: 1077,
         0.006896551724137931: 192,
         0.041379310344827586: 59,
         0.1103448275862069: 35,
         0.013793103448275862: 69,
         0.09655172413793103: 27,
         0.020689655172413793: 75,
         0.18620689655172412: 55,
         0.1793103448275862: 65,
         0.1724137931034483: 63,
         0.30344827586206896: 31,
         0.02413793103448276: 48,
         0.05862068965517241: 33,
         0.034482758620689655: 41,
         0.06896551724137931: 24,
         0.03793103448275862: 30,
         0.1896551724137931: 64,
         0.15517241379310345: 70,
         0.32413793103448274: 46,
         0.2655172413793103: 10,
         0.10344827586206896: 26,
         0.12413793103448276: 44,
         0.13448275862068965: 50,
         0.32068965517241377: 40,
         0.24137931034482757: 22,
         0.08275862068965517: 9,
         0.017241379310344827: 65,
         0.010344827586206896: 125,
         0.07586206896

In [83]:
# 合并归一化后的数据
mm_wp_df=pd.DataFrame(mm_wp)
mm_wp_df=mm_wp_df.set_index(M.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(M.index)
ML=pd.concat([wt_encoded_df,mm_wp_df,M['total_litter']],axis=1)

In [84]:
ML

,0,1,2,3,0,1,2,3,4,5,...,8,9,10,11,12,13,14,15,16,total_litter
2022-01-01,0.0,0.0,1.0,0.0,0.003448,0.000000,0.006897,0.041379,0.110345,0.013793,...,0.020690,0.186207,0.179310,0.172414,0.172414,0.303448,0.303448,0.041379,0.041379,57.0
2022-01-02,0.0,0.0,1.0,0.0,0.003448,0.006897,0.000000,0.024138,0.058621,0.034483,...,0.037931,0.189655,0.172414,0.172414,0.155172,0.324138,0.265517,0.103448,0.041379,48.0
2022-01-07,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.006897,0.000000,...,0.000000,0.155172,0.124138,0.134483,0.110345,0.320690,0.241379,0.082759,0.017241,2.0
2022-01-10,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.003448,0.124138,0.010345,...,0.003448,0.155172,0.110345,0.141379,0.103448,0.344828,0.279310,0.031034,0.006897,41.0
2022-01-11,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.006897,0.141379,0.110345,0.134483,0.110345,0.344828,0.262069,0.048276,0.020690,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,0.0,0.0,1.0,0.0,0.027586,0.100000,0.041379,0.231034,0.224138,0.003448,...,0.120690,0.165517,0.155172,0.165517,0.155172,0.344828,0.320690,0.041379,0.017241,229.0
2022-12-27,1.0,0.0,0.0,0.0,0.020690,0.168966,0.024138,0.013793,0.110345,0.013793,...,0.155172,0.158621,0.141379,0.148276,0.124138,0.320690,0.244828,0.075862,0.031034,152.0
2022-12-29,0.0,0.0,1.0,0.0,0.013793,0.124138,0.013793,0.048276,0.103448,0.003448,...,0.231034,0.179310,0.155172,0.165517,0.141379,0.320690,0.262069,0.100000,0.051724,160.0
2022-12-30,0.0,0.0,1.0,0.0,0.100000,0.103448,0.041379,0.100000,0.131034,0.003448,...,0.275862,0.186207,0.134483,0.179310,0.134483,0.344828,0.279310,0.100000,0.041379,228.0


In [84]:
# ML_array=np.array(ML)

In [87]:
# ML.to_csv('./outdata/Krommenie_train.csv')

## poi提取(舍弃）

In [164]:
poi1=pd.read_csv('./outdata/poi1.csv')

In [165]:
poi1['geometry']

0      POINT (5.0817843 52.6456291)
1      POINT (5.0961325 52.6457239)
2      POINT (5.0812344 52.6458317)
3      POINT (5.0968648 52.6460559)
4      POINT (5.0853898 52.6531698)
                   ...             
450    POINT (5.0931458 52.6533975)
451    POINT (5.0932666 52.6531166)
452    POINT (5.0932798 52.6532018)
453    POINT (5.0934684 52.6531761)
454    POINT (5.0936391 52.6532932)
Name: geometry, Length: 455, dtype: object

In [166]:
points['geometry'].astype(str)

0       POINT (5.0624231 52.6350804)
1       POINT (5.0085054 52.6350184)
2         POINT (5.0578571 52.63695)
3       POINT (5.0572411 52.6381662)
4       POINT (5.0523931 52.6417891)
                    ...             
2397     POINT (5.0472634 52.647675)
2398    POINT (5.0469239 52.6475957)
2399    POINT (5.0465883 52.6476558)
2400    POINT (5.0473351 52.6479847)
2401    POINT (5.0471709 52.6476163)
Name: geometry, Length: 2402, dtype: object

In [167]:
poi1_flag=points['geometry'].astype(str).isin(poi1['geometry'])

In [168]:
poi1_flag.value_counts()

False    1947
True      455
Name: geometry, dtype: int64

In [169]:
poi1=points[poi1_flag].reset_index(drop=True)# 第一类poi

In [170]:
poi1.to_file('./poi1.shp')# 输出为shapefile文件

In [171]:
poi1[['osm_id','type','geometry']].set_index(['osm_id'])

,type,geometry
osm_id,,
47077200,traffic,POINT (5.08178 52.64563)
47080156,traffic,POINT (5.09613 52.64572)
47081177,traffic,POINT (5.08123 52.64583)
47081536,traffic,POINT (5.09686 52.64606)
47097804,traffic,POINT (5.08539 52.65317)
...,...,...
49020516,miscpoi,POINT (5.09315 52.65340)
49020522,miscpoi,POINT (5.09327 52.65312)
49020523,miscpoi,POINT (5.09328 52.65320)


## 时间分段标签

In [85]:
df_date=pd.to_datetime(df_dateconcat.index)

In [86]:
df_date

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-07', '2022-01-10',
               '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-15',
               '2022-01-17', '2022-01-18',
               ...
               '2022-12-18', '2022-12-19', '2022-12-21', '2022-12-23',
               '2022-12-24', '2022-12-25', '2022-12-27', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', name='datetime', length=249, freq=None)

In [87]:
is_weekly=df_date.weekday==(5 or 6) # 一周结束

In [88]:
df_date.weekday==(5 or 6)

array([ True, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False,  True, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,  True, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False,

In [89]:
holiday=pd.to_datetime([
        '2022-01-01','2022-04-02','2022-04-04','2022-04-17','2022-04-18','2022-04-27','2022-05-04','2022-05-05','2022-05-26','2022-06-05','2022-06-06',
         '2022-12-05','2022-12-06','2022-12-24','2022-12-25','2022-12-26','2022-12-31'])

In [90]:
is_holiday=df_date.isin(holiday)

In [91]:
is_holiday=np.array(is_holiday)+np.array(df_date.weekday==(5 or 6))

In [92]:
Date2=pd.DataFrame(np.zeros([249,3]),columns=['daily','weekly','holiday'])

In [93]:
Date2=Date2.set_index(df_date.date)

In [94]:
ML1=pd.concat([Date2,ML],axis=1)

In [95]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,8,9,10,11,12,13,14,15,16,total_litter
2022-01-01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.003448,0.000000,0.006897,...,0.020690,0.186207,0.179310,0.172414,0.172414,0.303448,0.303448,0.041379,0.041379,57.0
2022-01-02,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.003448,0.006897,0.000000,...,0.037931,0.189655,0.172414,0.172414,0.155172,0.324138,0.265517,0.103448,0.041379,48.0
2022-01-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.155172,0.124138,0.134483,0.110345,0.320690,0.241379,0.082759,0.017241,2.0
2022-01-10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.003448,0.155172,0.110345,0.141379,0.103448,0.344828,0.279310,0.031034,0.006897,41.0
2022-01-11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.006897,0.141379,0.110345,0.134483,0.110345,0.344828,0.262069,0.048276,0.020690,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.027586,0.100000,0.041379,...,0.120690,0.165517,0.155172,0.165517,0.155172,0.344828,0.320690,0.041379,0.017241,229.0
2022-12-27,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020690,0.168966,0.024138,...,0.155172,0.158621,0.141379,0.148276,0.124138,0.320690,0.244828,0.075862,0.031034,152.0
2022-12-29,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.013793,0.124138,0.013793,...,0.231034,0.179310,0.155172,0.165517,0.141379,0.320690,0.262069,0.100000,0.051724,160.0
2022-12-30,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.100000,0.103448,0.041379,...,0.275862,0.186207,0.134483,0.179310,0.134483,0.344828,0.279310,0.100000,0.041379,228.0


In [96]:
ML1['daily']=1.0

In [97]:
ML1[is_weekly].index

Index([2022-01-01, 2022-01-15, 2022-01-22, 2022-02-05, 2022-02-12, 2022-02-26,
       2022-03-05, 2022-03-12, 2022-03-19, 2022-03-26, 2022-04-02, 2022-04-09,
       2022-04-16, 2022-04-23, 2022-04-30, 2022-05-07, 2022-05-14, 2022-05-21,
       2022-05-28, 2022-06-04, 2022-06-11, 2022-06-18, 2022-06-25, 2022-07-02,
       2022-07-09, 2022-07-16, 2022-07-23, 2022-08-06, 2022-08-13, 2022-08-20,
       2022-09-03, 2022-09-10, 2022-09-17, 2022-09-24, 2022-10-01, 2022-10-08,
       2022-10-15, 2022-10-22, 2022-10-29, 2022-11-12, 2022-11-19, 2022-11-26,
       2022-12-03, 2022-12-10, 2022-12-17, 2022-12-24, 2022-12-31],
      dtype='object')

In [98]:
ML1.loc[is_weekly,'weekly']=1.0

In [99]:
ML1['weekly'].value_counts()

0.0    202
1.0     47
Name: weekly, dtype: int64

In [100]:
ML1.loc[is_holiday,'holiday']=1.0

In [101]:
ML1['holiday'].value_counts()

0.0    192
1.0     57
Name: holiday, dtype: int64

In [102]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,8,9,10,11,12,13,14,15,16,total_litter
2022-01-01,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.003448,0.000000,0.006897,...,0.020690,0.186207,0.179310,0.172414,0.172414,0.303448,0.303448,0.041379,0.041379,57.0
2022-01-02,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.003448,0.006897,0.000000,...,0.037931,0.189655,0.172414,0.172414,0.155172,0.324138,0.265517,0.103448,0.041379,48.0
2022-01-07,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.155172,0.124138,0.134483,0.110345,0.320690,0.241379,0.082759,0.017241,2.0
2022-01-10,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.003448,0.155172,0.110345,0.141379,0.103448,0.344828,0.279310,0.031034,0.006897,41.0
2022-01-11,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.006897,0.141379,0.110345,0.134483,0.110345,0.344828,0.262069,0.048276,0.020690,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-25,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.027586,0.100000,0.041379,...,0.120690,0.165517,0.155172,0.165517,0.155172,0.344828,0.320690,0.041379,0.017241,229.0
2022-12-27,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.020690,0.168966,0.024138,...,0.155172,0.158621,0.141379,0.148276,0.124138,0.320690,0.244828,0.075862,0.031034,152.0
2022-12-29,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.013793,0.124138,0.013793,...,0.231034,0.179310,0.155172,0.165517,0.141379,0.320690,0.262069,0.100000,0.051724,160.0
2022-12-30,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.100000,0.103448,0.041379,...,0.275862,0.186207,0.134483,0.179310,0.134483,0.344828,0.279310,0.100000,0.041379,228.0


In [103]:
ML1.to_csv('./outdata/Krommenie_ML.csv')